In [1]:
from gensim.models import word2vec
import numpy as np
from scipy import spatial

# Import & Init jieba
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re

Using TensorFlow backend.
Building prefix dict from /home/sunset/word_contest/datas/dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u849ecfdca27003d306f39ca004b82b5b.cache
Loading model cost 1.170 seconds.
Prefix dict has been built succesfully.


### Load datasets

In [2]:
sample = pd.read_csv('datas/sample_test_data.txt')
sample

,id,dialogue,options,answer
0,0,A:你這麼快就知道了,B:全家就是你家\tB:付出不是浪費時間\tB:願意為社會付出的人太少了\tB:我都是一個人...,4
1,1,A:每一支冰塊都不同 水質不同 硬度不同,A:紋路不同\tA:也是你唯一發洩情緒的辦法吧\tA:還記不記得那次你在我家巷口\tA:像拼...,0
2,2,A:這樣沖這麼一大塊網子 要多久時間,B:有更舒適的環境\tB:感覺是一隻很貼心的貓咪\tB:我相信一定可以成功的啦\tB:可以講...,5
3,3,A:兒子啊 都幾點了 你還不睡,B:沒關係啦 我來幫你\tB:我在背書啦 就快背好了\tB:造成誤會 不是很可惜嗎\tB:女...,1
4,4,A:孩子還在念書的時候 看到其他小朋友都有父母 而自己沒有媽媽,A:那孩子就是突然\tA:就是說\tA:禮拜一到學校\tA:媽媽還不會開車\tA:就會問我媽...,4
5,5,A:那牠吐絲這三天三夜\tA:你們不就都要一直在這邊守護著牠囉,B:涂先生\tB:那這些蠶寶寶在這邊吐絲\tB:要吐多久啊\tB:溫度三十多度的話\tB:對...,4
6,6,A:奇怪了 濂僑跑到哪兒去了呢 濂僑 你怎麼啦,B:我可能吃壞肚子了 肚子好痛喔\tB:另外一種比法是 一星期有七天\tB:為什麼不可以 同...,0
7,7,A:是什麼讓玉屏的改變這麼大啊,B:當然是紀老師的魔法囉\tB:都交給這位大力士吧\tB:粗重的交給我就對了\tB:我是八年...,0
8,8,A:曉書啊 你手上拿的 該不是最新的遊戲機吧,B:你不要小看這個扯鈴\t B:互相幫忙一下嘛\tB:一個週末出去走走 沒有關係的啦\tB:...,3
9,9,A:失戀的確會讓人家很難過,A:所以你要縮短認真難過的時間\tA:他走了十三個年頭\tA:獲選之後當然是很開心囉\tA:...,0


In [7]:
x1 = [[ for s in dialog.split('\t')] for dialog in sample.dialogue.values]
x2 = [[jieba.cut(s[2:]) for s in option.split('\t')] for option in sample.options.values]
y = sample.answer.values

In [15]:
x1[7], x2[7], y[7]

([['是', '什麼', '讓', '玉屏', '的', '改變', '這麼', '大', '啊']],
 [['當然', '是紀', '老師', '的', '魔法', '囉'],
  ['都', '交給', '這位', '大力士', '吧'],
  ['粗重', '的', '交給', '我', '就', '對', '了'],
  ['我', '是', '八年', '八班', '來', '拿', '講義', '的'],
  ['玉屏', '一個', '人', '怎麼', '搬得動', '呢'],
  ['就是', '你', '剛剛', '幫', '老師', '拿', '作業本', '的', '時候']],
 0)

In [17]:
max(len(q) for q in x1)

3

In [18]:
x1

[[['你', '這麼', '快', '就', '知道', '了']],
 [['每', '一支', '冰塊', '都', '不同', ' ', '水質', '不同', ' ', '硬度', '不同']],
 [['這樣', '沖', '這麼', '一大塊', '網子', ' ', '要', '多久', '時間']],
 [['兒子', '啊', ' ', '都', '幾點', '了', ' ', '你', '還不睡']],
 [['孩子',
   '還在',
   '念書',
   '的',
   '時候',
   ' ',
   '看到',
   '其他',
   '小朋友',
   '都',
   '有',
   '父母',
   ' ',
   '而',
   '自己',
   '沒有',
   '媽媽']],
 [['那', '牠', '吐絲', '這', '三天三夜'],
  ['你們', '不就', '都', '要', '一直', '在', '這邊', '守護', '著', '牠', '囉']],
 [['奇怪',
   '了',
   ' ',
   '濂',
   '僑',
   '跑',
   '到',
   '哪兒',
   '去',
   '了',
   '呢',
   ' ',
   '濂',
   '僑',
   ' ',
   '你',
   '怎麼',
   '啦']],
 [['是', '什麼', '讓', '玉屏', '的', '改變', '這麼', '大', '啊']],
 [['曉書',
   '啊',
   ' ',
   '你',
   '手上',
   '拿',
   '的',
   ' ',
   '該',
   '不是',
   '最新',
   '的',
   '遊戲機',
   '吧']],
 [['失戀', '的確', '會', '讓', '人家', '很', '難過']],
 [['漁網', '破', '了', ' ', '魚跑', '了', ' ', '漁民', '的', '臉', '就', '綠', '了']],
 [['可以', '穿', '襪子', '了'], ['怎麼樣']],
 [['下', '下禮拜', '就要', '段考', '了', ' ', '你們', '都', '不用', '看書', '

### word2vec model

In [ ]:
word2vec_model = word2vec.Word2Vec.load('models/word2vec_250.model.bin')

In [ ]:
# Numbers of words in dictionary
len(word2vec_model.wv.vocab)